In [1]:
import pandas as pd
from textblob import TextBlob   
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
from sklearn.model_selection import train_test_split
#import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
#import tensorflow as tf
#import keras
import numpy as np
import pandas as pd
print('Done')

ModuleNotFoundError: No module named 'gensim'

In [6]:
!git clone https://github.com/RaRe-Technologies/gensim.git
!cd gensim
!pip install -e .


Cloning into 'gensim'...
error: RPC failed; curl 56 OpenSSL SSL_read: Connection was reset, errno 10054
error: 2251 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output
The system cannot find the path specified.
ERROR: File "setup.py" not found. Directory cannot be installed in editable mode: C:\Users\User\Desktop\capstone


In [4]:
dfn = pd.read_csv("nikol.csv")

In [12]:
dfn.dropna()
dfn['Tweets'] = dfn['Tweets'].str.replace('https', '')

dfn['polarity'] = dfn['Tweets'].apply(lambda x: TextBlob(x).polarity)

label = []
for pol in dfn['polarity']:
    if 0 < pol <= 1.0 :    label.append('positive')
    elif -1.0 <= pol < 0:   label.append('negative')
    elif pol == 0:  label.append('neutral')
        
        
dfn['label'] = label

In [14]:
dfe = pd.read_csv('marukyan.csv')
dfe.dropna()
dfe['Tweets'] = dfe['Tweets'].str.replace('https', '')

dfe['polarity'] = dfe['Tweets'].apply(lambda x: TextBlob(x).polarity)

label = []
for pol in dfe['polarity']:
    if 0 < pol <= 1.0 :    label.append('positive')
    elif -1.0 <= pol < 0:   label.append('negative')
    elif pol == 0:  label.append('neutral')
        
        
dfe['label'] = label

In [16]:
dfa = pd.read_csv('mirzoyan.csv')
dfa.dropna()
dfa['Tweets'] = dfa['Tweets'].str.replace('https', '')

dfa['polarity'] = dfa['Tweets'].apply(lambda x: TextBlob(x).polarity)

label = []
for pol in dfa['polarity']:
    if 0 < pol <= 1.0 :    label.append('positive')
    elif -1.0 <= pol < 0:   label.append('negative')
    elif pol == 0:  label.append('neutral')
        
        
dfa['label'] = label

In [17]:
all = pd.concat([dfn, dfe,dfa])

In [21]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(all, test_size=0.2)

In [24]:
train['label'].unique()

array(['positive', 'negative', 'neutral'], dtype=object)

In [25]:
train.groupby('label').nunique()

,Date Created,Number of Likes,Source of Tweet,Tweets,polarity,label
label,,,,,,
negative,234,128,4,237,114,1
neutral,1125,265,4,1214,1,1
positive,689,275,4,712,312,1


In [26]:
train = train[['Tweets','label']]
train.head()

,Tweets,label
1144,Dear Parliamentarians of the Council of Europe...,positive
216,@NikolPashinyan sent a condolence message to #...,negative
1421,📌 * Notice. The discussion was held in compli...,neutral
1681,Today in the Parliament we stopped the passing...,positive
497,ՌԴ ՊՆ պաշտոնական հեռուստաալիքն ընդհանրացրել է ...,neutral


In [28]:
train["Tweets"].isnull().sum()

0

In [29]:
def depure_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

In [42]:
temp = []
#Splitting pd.Series to list
data_to_list = train['Tweets'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
list(temp[:5])

['Dear Parliamentarians of the Council of Europe, Dear Colleagues, As you already know Azerbaijan has launched a full scale war against the Nagorno Karabakh Republic early on September 27.',
 'sent a condolence message to #German Chancellor Angela #Merkel on the occasion of the tragic incident in Hanau town.',
 '📌 * Notice. The discussion was held in compliance with all the rules set by the commandant’s office regarding the activities of open-air cafes.',
 'Today in the Parliament we stopped the passing of the law that directly violated rights of Armenian citizens. Nevertheless, the ruling party is trying to pass the same law once again without notifying us. This is a warning call of authoritarianism against which we have fought.',
 'ՌԴ ՊՆ պաշտոնական հեռուստաալիքն ընդհանրացրել է Նժդեհին, ֆաշիզմը և ՀՀԿ-ն. Տեսանյութ ://t.co/dp8UKKoI9u via ']

In [50]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

data_words = list(sent_to_words(temp))

print(data_words[:10])

NameError: name 'gensim' is not defined